<a href="https://colab.research.google.com/github/LucasCabra7/Project-WEBSCRAPING/blob/main/C%C3%B3pia_de_OPENSENSE_Lucas_Cabral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O objetivo é criar uma aplicação simples (WEBSCRAPING) em Python que receba uma pasta compactada no formato .zip, contendo um conjunto de arquivos HTML. A aplicação deve descompactar o arquivo .zip e, para cada arquivo HTML processado, extrair uma tabela com as seguintes features (características):


*   Nome do Arquivo
*   CNPJ
*   RESULTADO

SE HOUVER RESULTADOS:

*   NÚMERO DO PEDIDO
*   DATA DO DEPÓSITO
*   TÍTULO
*   IPC











## 1° Descompactar o arquivo zipado:

In [ ]:
import zipfile as zf

In [ ]:
# Extrair os arquivos HTML do zip e armazenar em uma nova "pasta"
pasta_html = "PATENTES"
with zf.ZipFile('Opensense-proficiency-PATENTES.zip', 'r') as zip_object:
  zip_object.extractall(pasta_html)

## 2° Processar os arquivos HTML, verificando o encoding e lendo o arquivo:

In [ ]:
import os
import re
import io
import chardet
from bs4 import BeautifulSoup

In [ ]:
dados = []

for arquivo in os.listdir(pasta_html):
  if arquivo.endswith(".html"):
    caminho = os.path.join(pasta_html, arquivo)

    # lendo o conteúdo do arquivo
    with open(caminho, 'rb') as f:
      leitura = f.read()
      encoding_verificado = chardet.detect(leitura)["encoding"]
      conteudo_html = leitura.decode(encoding_verificado)

    # 3° utilizaremos a biblioteca BeautifulSoup para extrair automaticamente os dados necessários das páginas HTML (Web), que servirão como base para a criação das features.
    soup = BeautifulSoup(conteudo_html, "html.parser")

    # Extrair o CNPJ do arquivo HTML:
    cnpj = None # Incializar com NONE para caso não seja encontrado

    for fonte in soup.find_all("font", class_="normal"):
      texto = fonte.get_text(strip=True)
      if "CNPJ" in texto.upper():
        cnpj_qtd = re.search(r"(\d{11,14})", texto) # Procurar numeros com 11 a 14 digitos - padrão cpf/cnpj
        if cnpj_qtd:
          cnpj = cnpj_qtd.group(1)
          break

    # Extrair o Resultado e verificando se o Resultado é igual a "0"
    texto_vazio = soup.get_text().lower()
    if "nenhum resultado" in texto_vazio or "0 processos" in texto_vazio:
      dados.append(
          {
              "Arquivo": arquivo,
              "CNPJ": cnpj,
              "RESULTADO": 0,
              "NÚMERO DO PEDIDO": None,
              "DATA DO DEPÓSITO": None,
              "Título": None,
              "IPC": None
          }
      )
    else:
      ## Caso contrário, verificar a quantidade de resultados
      tabelas = soup.find_all("table")
      if len(tabelas) > 1:
        tabela_resultados = tabelas[1]
      elif len(tabelas) == 1:
        tabela_resultados = tabelas[0]
      else:
        tabela_resultados = None

      # Verificando se o resultado é diferente de 0
      if tabela_resultados:
        linhas_tabela = tabela_resultados.find_all("tr")[1:]
        num_processos = len(linhas_tabela)

        # Iterar sobre as linhas da tabela
        for linha in linhas_tabela:
          colunas = linha.find_all("td")

          # Verificando e acessando as colunas do Resultado:
          if len(colunas) >= 4:
            numero_pedido = colunas[0].text.strip()
            data_deposito = colunas[1].text.strip()
            titulo = colunas[2].text.strip()
            ipc = colunas[3].text.strip()
            dados.append(
                {
                    "Arquivo": arquivo,
                    "CNPJ": cnpj,
                    "RESULTADO": num_processos,
                    "NÚMERO DO PEDIDO": numero_pedido,
                    "DATA DO DEPÓSITO": data_deposito,
                    "Título": titulo,
                    "IPC": ipc
                }
            )

## 4° Gerar um Data Frame com nossos dados coletados:

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(dados)
df

,Arquivo,CNPJ,RESULTADO,NÚMERO DO PEDIDO,DATA DO DEPÓSITO,Título,IPC
0,00011009000106-01.html,00011009000106,0,None,None,None,None
1,00020777000118-01.html,00020777000118,0,None,None,None,None
2,00015174000128-01.html,00015174000128,0,None,None,None,None
3,00000000000191-01.html,00000000000191,11,Pedido,Dep�sito,T�tulo,IPC
4,00000000000191-01.html,00000000000191,11,BR 10 2018 071855 0,24/10/2018,,-
5,00000000000191-01.html,00000000000191,11,BR 20 2014 014852 9,17/06/2014,CONFIGURA��O APLICADA EM TERMINAL DE AUTOATEND...,E04H 1/12
6,00014522000142-01.html,00014522000142,12,Pedido,Depósito,Título,IPC
7,00014522000142-01.html,00014522000142,12,BR 10 2016 025387 0,31/10/2016,PROCESSO APLICADO NA OBTENÇÃO DE UM PRODUTO ES...,A61K 8/34
8,00014522000142-01.html,00014522000142,12,BR 10 2015 006845 0,26/03/2015,,-
9,00014522000142-01.html,00014522000142,12,BR 10 2013 025581 5,03/10/2013,,-
